# API travel recommandation

Pour pouvoir utiliser mon api j'ai du ecrire sur ma ligne de commande pip install amadeus

Mon api est une api d'IA car elle se base sur les données accumulées par Amadeus sur les vols antérieurs pour prédire en fonction des mêmes profiles, les villes que les gens pourraient aimer 

Cette API recommande des destinations de voyage avec comme pays de départ: la France,  l'Allemagne, l'Italie, l'Espagne, les Pays-bas, l'USA, l'Argentine et  l'Arabie saoudite.

In [10]:
from dotenv import load_dotenv 
load_dotenv()
import os 

api_keys = os.getenv("api_keys")
api_secret=os.getenv("api_secret")

/home/el-brek/miniconda3/envs/django_api/bin:/home/el-brek/miniconda3/condabin:/home/el-brek/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


# QUELQUES REQUETES POUR COMPRENDRE L'API

In [11]:
response = amadeus.shopping.flight_offers_search.get(
        originLocationCode='MAD',
        destinationLocationCode='ATH',
        departureDate='2022-11-01',
        adults=1)
billet = response.result
#print(response.data)
#for i in range(3):
#    vol_aller= billet[i]["itineraries"]["departure"]["at"]

In [12]:
from amadeus import Client, ResponseError

amadeus = Client(
    client_id=api_keys,
    client_secret=api_secret
)

In [13]:
interet = amadeus.reference_data.locations.points_of_interest.get(
    latitude = 48.85341,
    longitude = 2.3488
)
interet = interet.result 
centre_dinteret = []
for i in range(3) :
    interets = interet["data"][i]["name"]
    centre_dinteret.append(interets) 
centre_dinteret

['Cathédrale Notre-Dame de Paris', 'Centre Pompidou', 'Île de la Cité']

In [14]:
#che=amadeus.reference_data.urls.checkin_links.get(airlineCode='BA')
#che.data


In [101]:
#exemple1 = amadeus.travel.predictions.trip_purpose.get(originLocationCode='ATH', destinationLocationCode='MAD', departureDate='2022-11-01', returnDate='2022-11-08')
#exemple1.result

In [51]:
#amadeus.get('/v2/reference-data/urls/checkin-links', airlineCode='BA')

In [7]:
from amadeus import Location

response = amadeus.reference_data.locations.get(
    keyword='LON',
    subType=Location.ANY
)

In [21]:

#exemple2 = amadeus.reference_data.recommended_locations.get(cityCodes='MAD', travelerCountryCode='FR')
#exemple2.data

In [100]:
#exemple2 = amadeus.reference_data.recommended_locations.get(cityCodes='LYS,PAR,MAD,LIS,ALG', travelerCountryCode='FR')
#exemple2.body

In [8]:
exemple2 = amadeus.reference_data.recommended_locations.get(cityCodes='LYS', travelerCountryCode='FR')
exemple2.result

{'data': [{'subtype': 'CITY',
   'name': 'Paris',
   'iataCode': 'PAR',
   'geoCode': {'latitude': 48.85341, 'longitude': 2.3488},
   'type': 'recommended-location',
   'relevance': 0.98025961},
  {'subtype': 'CITY',
   'name': 'Marseille',
   'iataCode': 'MRS',
   'geoCode': {'latitude': 43.435559999999995, 'longitude': 5.21361},
   'type': 'recommended-location',
   'relevance': 0.93377978},
  {'subtype': 'CITY',
   'name': 'Genève',
   'iataCode': 'GVA',
   'geoCode': {'latitude': 46.238064, 'longitude': 6.10895},
   'type': 'recommended-location',
   'relevance': 0.93232902},
  {'subtype': 'CITY',
   'name': 'Tunis',
   'iataCode': 'TUN',
   'geoCode': {'latitude': 36.85103, 'longitude': 10.227219999999999},
   'type': 'recommended-location',
   'relevance': 0.81634752},
  {'subtype': 'CITY',
   'name': 'Nantes',
   'iataCode': 'NTE',
   'geoCode': {'latitude': 47.153189000000005, 'longitude': -1.610725},
   'type': 'recommended-location',
   'relevance': 0.81509179}],
 'meta': {'c

# FACTORISER NOTRE REQUETE


In [13]:

client_un = { "ville" : ["MAD","PAR"], "pays" : "FR"}
client_un["ville"]


['MAD', 'PAR']

Après avoir testé, mon api : j'ai décidé de l'utiliser en demandant à mon client, plusieurs noms de ville qu'il aime bien (jusqu'à 5 villes), le pays de départ et ses dates de voyage. A la suite de cela mon code va transformer les noms en IATA code. Puis va sortir 3 destinations qui pourrait l'interresser avec 3 lieu interessant à visiter.

In [46]:
latitudes=[]
longitudes=[]
preference = amadeus.reference_data.recommended_locations.get(cityCodes ="MAD", travelerCountryCode = "FR")
preference= preference.result
print(preference)
#for i in range(3):
#   latitude = preference["data"][i]["geoCode"]["latitude"]
#    longitude = preference["data"][i]["geoCode"]["longitude"]
#    latitudes.append(latitude)
#    longitudes.append(longitude)


{'data': [{'subtype': 'CITY', 'name': 'Barcelona', 'iataCode': 'BCN', 'geoCode': {'latitude': 41.297078000000006, 'longitude': 2.078464}, 'type': 'recommended-location', 'relevance': 0.97059254}, {'subtype': 'CITY', 'name': 'Lisbon', 'iataCode': 'LIS', 'geoCode': {'latitude': 38.78131, 'longitude': -9.13592}, 'type': 'recommended-location', 'relevance': 0.90305743}, {'subtype': 'CITY', 'name': 'Rome', 'iataCode': 'ROM', 'geoCode': {'latitude': 41.89193, 'longitude': 12.511330000000001}, 'type': 'recommended-location', 'relevance': 0.88211507}, {'subtype': 'CITY', 'name': 'Sevilla', 'iataCode': 'SVQ', 'geoCode': {'latitude': 37.418, 'longitude': -5.89311}, 'type': 'recommended-location', 'relevance': 0.86900225}, {'subtype': 'CITY', 'name': 'Porto', 'iataCode': 'OPO', 'geoCode': {'latitude': 41.248055, 'longitude': -8.681389}, 'type': 'recommended-location', 'relevance': 0.85095064}], 'meta': {'count': 5, 'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/recommended-loca

In [49]:
from amadeus import Client, ResponseError
def preference_voyage(information_client):
    amadeus = Client(client_id=api_keys,
    client_secret = api_secret)

    from amadeus import Location
    param_client = {}
    for key,value in information_client.items():
        param_client.update({"cityCodes":key, "travelerCountrycode":value})
    
    preference = amadeus.reference_data.recommended_locations.get(cityCodes = information_client["ville"], travelerCountryCode = information_client["pays"])
    #preference = amadeus.reference_data.recommended_locations.get(params = param_client)
    preference=preference.result
    print("Voici les villes qui pourrait vous interesser:")
    #villes_destination=[]
    #latitudes=[]
    #longitudes=[]
    #centre_dinteret = []
    for i in range(3):
        latitudes = preference["data"][i]["geoCode"]["latitude"]
        longitudes = preference["data"][i]["geoCode"]["longitude"]
        print(latitudes)
        print(longitudes)
       
        nom_de_villes = preference["data"][i]["name"]
        print( nom_de_villes)
        #centre_dinteret = []
        #for i in range(3) :
           # interets = interet["data"][i]["name"]
            #centre_dinteret.append(interets) 
        #print(centre_dinteret)

       
       
        #villes_destination.append(nom_de_villes)
        #interet = amadeus.reference_data.locations.points_of_interest.get(latitude = latitudes,longitude = 43.435559999999995)
        #print("Voici quelque endroit à découvrir dans cette ville:")
        #for j in range(3):
            #interets = interet["data"][j]["name"]
    
            #print(interets)    
    
    

    #return villes_destination
    
   

In [50]:
preference_voyage(client_un)





Voici les villes qui pourrait vous interesser:
43.435559999999995
5.21361
Marseille
45.72639
5.09083
Lyon
36.6985
3.2067200000000002
Algiers


Je n'arrive pas à factoriser en prenant la longitude et latitude de chaque ville pour donner les lieux à visiter je travaille encore là dessus

Je pense aussi à améliorer mon API en proposant aussi les billets d'avion ou proposer des photos avec une autre ou encore donner la meteo dans les villes proposer